# DRUMS - COVASIM Data Generation

In [7]:
import sys
sys.path.append('../')
# from drums_data_gen import *
from drums_data_gen_multi import *
import Modules.Loaders.DataFormatter as DF
from Modules.Utils.Imports import *

In [10]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

Device set to cpu


- `drums_gata_gen` generates data for one simulation.
- `drums_data_gen_multi` generates data for multiple simulations and averages over the results.

***

- `test_prob`: This is a float value that corresponds to the probability of testing given the agent is symptomatic. Currently, the values for proabbility of testing for other scenarios are `asymp_prob=0.001`, `symp_quar_prob=0.3`, and `asymp_quar_prob=0.3`. Additionally, the policy for quarantining is `quar_policy='daily'`
- `trace_lb` and `trace_ub`: These float values correspond to the lower and upper bounds on probability of tracing. Currently, the trace probability for the layers are `h=1.0`, `s=0.5`, `w=0.5`, `c=0.3`.
- `chi_type`: The type of the function that interacts with the tracing probability. Can choose between `constant`, `linear`, `piecewise`, and `sin`.
- `dynamic`: Boolean value that indicates whether or not to include the interacting term/function.
- `keep_d`: Boolean value that indicates wehter or not to include the D - diagnosed compartments.
- `model_params:` Dictionary containing keys pointing to parameter values, the compartment model's values over the simulation, and more.

In [2]:
population = int(200e3)
test_prob = 0.1
trace_lb = 0.0
trace_ub = 0.3
chi_type = 'piecewise'
keep_d = True
dynamic = True
maskb = True
n_runs = 10
model_params = ModelParams(population, test_prob, trace_lb, trace_ub, chi_type, keep_d, dynamic, maskb)

In [3]:
# drums_data_generator(model_params)
drums_data_generator_multi(model_params, n_runs)

Initializing sim with 200000 people for 182 days
  Running "Sim 0": 2020-02-01 ( 0/182) (1.82 s)  ———————————————————— 1%
  Running "Sim 0": 2020-02-11 (10/182) (2.72 s)  •——————————————————— 6%
  Running "Sim 0": 2020-02-21 (20/182) (3.87 s)  ••—————————————————— 11%
  Running "Sim 0": 2020-03-02 (30/182) (5.03 s)  •••————————————————— 17%
  Running "Sim 0": 2020-03-12 (40/182) (6.40 s)  ••••———————————————— 22%
  Running "Sim 0": 2020-03-22 (50/182) (7.91 s)  •••••——————————————— 28%
  Running "Sim 0": 2020-04-01 (60/182) (9.36 s)  ••••••—————————————— 33%
  Running "Sim 0": 2020-04-11 (70/182) (10.84 s)  •••••••————————————— 39%
  Running "Sim 0": 2020-04-21 (80/182) (12.26 s)  ••••••••———————————— 44%
  Running "Sim 0": 2020-05-01 (90/182) (13.78 s)  •••••••••——————————— 50%
  Running "Sim 0": 2020-05-11 (100/182) (15.33 s)  •••••••••••————————— 55%
  Running "Sim 0": 2020-05-21 (110/182) (16.66 s)  ••••••••••••———————— 61%
  Running "Sim 0": 2020-05-31 (120/182) (17.92 s)  •••••••

In [15]:
path = '../Data/covasim_data/drums_data/'
retrain = False
case_name = get_case_name(population, test_prob, trace_ub, keep_d, dynamic=dynamic, chi_type=chi_type)

# params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name, plot=True)
params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name + '_' + str(n_runs), plot=True)

In [16]:
# split into train/val and convert to torch
# data = params['data']
data = np.mean(params['data'], axis=0)
# data = (data / params['population']).to_numpy()
data = (data / params['population'])
params.pop('data')
N = len(data)
split = int(0.8*N)
p = np.random.permutation(N)
x_train = to_torch(p[:split][:, None]/(N-1))
y_train = to_torch(data[p[:split]])
x_val = to_torch(p[split:][:, None]/(N-1))
y_val = to_torch(data[p[split:]])
t = np.arange(N)[:,None]

In [18]:
# rescale data values to represent number of people in state X rather than ratios
data *= population

plot=True
if plot:
    # data = params['data']
    n = data.shape[1]
    col_names = list('STEAYDQRF') if keep_d else list('STEAYQRF')
    # t = np.arange(1, data.shape[0] + 1)
    # plot compartments
    fig = plt.figure(figsize=(15, 15))
    for i in range(1, n + 1):
        ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
        ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
        ax.set_title(col_names[i - 1])
        ax.legend(fontsize=8)
        fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
        plt.tight_layout(pad=2)
        plt.savefig(os.path.join(path, case_name + '_' + str(n_runs) + '_avg' + '.png') )
        plt.show()

C:\Users\Teddy\AppData\Local\Temp\ipykernel_16292\2473311774.py:20: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
